# 🚀 Enhanced US Stock News Analyzer

**Features:**
- 🎨 Beautiful formatting with colors and gradients
- 📊 Interactive stock price charts (candlestick + volume)
- 🔗 AI summaries with referenced news links
- 📈 Real-time price data and market metrics

**Stocks analyzed: TSLA, TSM (customizable)**

In [1]:
# Install required packages (run once)
#!pip install openai feedparser requests python-dotenv yfinance matplotlib plotly nbformat

In [2]:
# Import libraries
import feedparser
import requests
from openai import OpenAI
from datetime import datetime, timedelta
import os
from dotenv import load_dotenv
import yfinance as yf
import matplotlib.pyplot as plt
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from IPython.display import display, HTML

# Load environment variables from .env file
load_dotenv()

True

In [3]:
# Initialize OpenAI client
api_key = os.getenv('OPENAI_API_KEY')

if not api_key:
    raise ValueError("OPENAI_API_KEY not found in .env file. Please add it to your .env file.")

client = OpenAI(api_key=api_key)
print("✓ OpenAI client initialized successfully!")

✓ OpenAI client initialized successfully!


## 📰 News Fetching Functions

In [4]:
def fetch_stock_news(ticker, num_articles=5):
    """
    Fetch latest news for a specific stock ticker from Google News RSS
    
    Args:
        ticker: Stock ticker symbol (e.g., 'TSLA', 'TSM')
        num_articles: Number of articles to fetch
    
    Returns:
        List of article dictionaries
    """
    url = f'https://news.google.com/rss/search?q={ticker}+stock&hl=en-US&gl=US&ceid=US:en'
    
    feed = feedparser.parse(url)
    
    articles = []
    for entry in feed.entries[:num_articles]:
        article = {
            'ticker': ticker,
            'title': entry.title,
            'link': entry.link,
            'published': entry.published if 'published' in entry else 'N/A',
            'summary': entry.summary if 'summary' in entry else entry.title
        }
        articles.append(article)
    
    return articles

## 📊 Stock Price Data Functions

In [5]:
def fetch_stock_price_data(ticker, period='1mo'):
    """
    Fetch historical stock price data
    
    Args:
        ticker: Stock ticker symbol
        period: Time period ('1d', '5d', '1mo', '3mo', '6mo', '1y', etc.)
    
    Returns:
        Dictionary with stock price data
    """
    stock = yf.Ticker(ticker)
    data = stock.history(period=period)
    
    # Get current info
    info = stock.info
    current_price = info.get('currentPrice', data['Close'].iloc[-1] if len(data) > 0 else 'N/A')
    
    return {
        'data': data,
        'current_price': current_price,
        'info': info
    }

In [6]:
def create_stock_chart(ticker, stock_data, period='1mo'):
    """
    Create an interactive stock price chart using Plotly
    
    Args:
        ticker: Stock ticker symbol
        stock_data: Stock data dictionary from fetch_stock_price_data
        period: Time period for the chart
    
    Returns:
        Plotly figure object
    """
    df = stock_data['data']
    current_price = stock_data['current_price']
    
    # Create candlestick chart
    fig = go.Figure(data=[go.Candlestick(
        x=df.index,
        open=df['Open'],
        high=df['High'],
        low=df['Low'],
        close=df['Close'],
        name='Price'
    )])
    
    # Add volume bar chart
    fig.add_trace(go.Bar(
        x=df.index,
        y=df['Volume'],
        name='Volume',
        yaxis='y2',
        opacity=0.3
    ))
    
    # Calculate price change
    start_price = df['Close'].iloc[0]
    end_price = df['Close'].iloc[-1]
    price_change = ((end_price - start_price) / start_price) * 100
    
    # Set color based on price change
    title_color = 'green' if price_change >= 0 else 'red'
    
    # Update layout
    fig.update_layout(
        title=f'<b>{ticker} Stock Price - {period.upper()}</b><br>' + 
              f'<span style="color:{title_color}; font-size:18px;">Current: ${current_price:.2f} ' +
              f'({price_change:+.2f}%)</span>',
        yaxis_title='Price ($)',
        yaxis2=dict(
            title='Volume',
            overlaying='y',
            side='right'
        ),
        xaxis_title='Date',
        height=500,
        hovermode='x unified',
        template='plotly_white'
    )
    
    return fig

## 🤖 AI Summary with References

In [7]:
def summarize_stock_news_with_references(ticker, articles):
    """
    Generate an AI summary with references to specific news articles
    
    Args:
        ticker: Stock ticker symbol
        articles: List of article dictionaries
    
    Returns:
        Dictionary with summary and referenced articles
    """
    # Prepare articles with numbering for reference
    news_text = "\n\n".join([
        f"[{i+1}] {article['title']}\nSource: {article['summary']}"
        for i, article in enumerate(articles)
    ])
    
    # Call OpenAI API with instruction to reference articles
    response = client.chat.completions.create(
        model="gpt-4o-mini",
        messages=[
            {
                "role": "system",
                "content": """You are a financial analyst assistant that summarizes stock news. 
                Provide concise, objective summaries highlighting key developments, market sentiment, and potential impacts.
                IMPORTANT: When referencing information, cite the article number in brackets like [1], [2], etc."""
            },
            {
                "role": "user",
                "content": f"""Please provide a brief summary of recent news for {ticker} stock based on these articles.
                When you mention specific information, reference the article number in brackets.
                
                Articles:
                {news_text}"""
            }
        ],
        max_tokens=600,
        temperature=0.7
    )
    
    return {
        'summary': response.choices[0].message.content,
        'articles': articles
    }

## 🎨 Beautiful Display Functions

In [8]:
def display_enhanced_analysis(ticker, summary_data, stock_data):
    """
    Display beautifully formatted stock analysis with HTML
    
    Args:
        ticker: Stock ticker symbol
        summary_data: Dictionary with summary and articles
        stock_data: Stock price data dictionary
    """
    current_price = stock_data['current_price']
    info = stock_data['info']
    
    # Calculate price change
    df = stock_data['data']
    start_price = df['Close'].iloc[0]
    end_price = df['Close'].iloc[-1]
    price_change = ((end_price - start_price) / start_price) * 100
    change_color = '#28a745' if price_change >= 0 else '#dc3545'
    change_symbol = '▲' if price_change >= 0 else '▼'
    
    # Company info
    company_name = info.get('longName', ticker)
    market_cap = info.get('marketCap', 'N/A')
    if market_cap != 'N/A':
        market_cap_formatted = f"${market_cap/1e9:.2f}B" if market_cap > 1e9 else f"${market_cap/1e6:.2f}M"
    else:
        market_cap_formatted = 'N/A'
    
    # Create HTML
    html = f"""
    <div style="font-family: 'Segoe UI', Tahoma, Geneva, Verdana, sans-serif; margin: 20px 0;">
        
        <!-- Header -->
        <div style="background: linear-gradient(135deg, #667eea 0%, #764ba2 100%); 
                    padding: 30px; border-radius: 15px; color: white; margin-bottom: 20px;">
            <h1 style="margin: 0; font-size: 36px; font-weight: bold;">{ticker}</h1>
            <p style="margin: 5px 0 0 0; font-size: 18px; opacity: 0.9;">{company_name}</p>
            <div style="margin-top: 20px;">
                <span style="font-size: 42px; font-weight: bold;">${current_price:.2f}</span>
                <span style="font-size: 24px; margin-left: 15px; color: {change_color};">
                    {change_symbol} {abs(price_change):.2f}%
                </span>
            </div>
            <div style="margin-top: 10px; font-size: 14px; opacity: 0.8;">
                Market Cap: {market_cap_formatted}
            </div>
        </div>
        
        <!-- AI Summary Section -->
        <div style="background: #f8f9fa; padding: 25px; border-radius: 10px; 
                    border-left: 5px solid #667eea; margin-bottom: 20px;">
            <h2 style="color: #333; margin-top: 0; font-size: 24px;">
                🤖 AI-Powered News Summary
            </h2>
            <div style="color: #555; line-height: 1.8; font-size: 16px;">
                {summary_data['summary'].replace('[', '<span style="background: #667eea; color: white; padding: 2px 6px; border-radius: 3px; font-size: 12px; font-weight: bold;">[').replace(']', ']</span>')}
            </div>
        </div>
        
        <!-- Referenced Articles -->
        <div style="background: white; padding: 25px; border-radius: 10px; 
                    box-shadow: 0 2px 10px rgba(0,0,0,0.1);">
            <h2 style="color: #333; margin-top: 0; font-size: 24px;">
                📰 Referenced News Articles
            </h2>
    """
    
    # Add articles with links
    for i, article in enumerate(summary_data['articles'], 1):
        html += f"""
            <div style="padding: 15px; margin: 10px 0; background: #f8f9fa; 
                        border-radius: 8px; border-left: 4px solid #667eea;">
                <div style="display: flex; align-items: center;">
                    <span style="background: #667eea; color: white; padding: 5px 10px; 
                                 border-radius: 5px; font-weight: bold; margin-right: 15px;">
                        [{i}]
                    </span>
                    <div style="flex: 1;">
                        <a href="{article['link']}" target="_blank" 
                           style="color: #333; text-decoration: none; font-size: 16px; font-weight: 500;">
                            {article['title']}
                        </a>
                        <div style="color: #888; font-size: 13px; margin-top: 5px;">
                            📅 {article['published']}
                        </div>
                    </div>
                </div>
            </div>
        """
    
    html += """
        </div>
    </div>
    """
    
    display(HTML(html))

## 🚀 Main Analysis Function

In [9]:
def analyze_stock_enhanced(ticker, num_articles=5, chart_period='1mo'):
    """
    Complete enhanced stock analysis with formatting, charts, and references
    
    Args:
        ticker: Stock ticker symbol
        num_articles: Number of news articles to fetch
        chart_period: Time period for price chart ('1d', '5d', '1mo', '3mo', '6mo', '1y')
    
    Returns:
        Dictionary with all analysis data
    """
    print(f"🔍 Analyzing {ticker}...\n")
    
    # Fetch news
    print(f"📰 Fetching latest news for {ticker}...")
    articles = fetch_stock_news(ticker, num_articles)
    print(f"✓ Found {len(articles)} articles\n")
    
    # Fetch stock data
    print(f"📊 Fetching stock price data...")
    stock_data = fetch_stock_price_data(ticker, chart_period)
    print(f"✓ Retrieved {chart_period} price data\n")
    
    # Generate AI summary with references
    print(f"🤖 Generating AI summary with references...")
    summary_data = summarize_stock_news_with_references(ticker, articles)
    print(f"✓ Summary generated\n")
    
    # Display enhanced analysis
    print("="*80)
    display_enhanced_analysis(ticker, summary_data, stock_data)
    
    # Display stock chart
    print("\n📈 Stock Price Chart:")
    fig = create_stock_chart(ticker, stock_data, chart_period)
    fig.show()
    
    return {
        'ticker': ticker,
        'articles': articles,
        'summary': summary_data,
        'stock_data': stock_data
    }

In [10]:
def compare_multiple_stocks_enhanced(tickers, num_articles=5, chart_period='1mo'):
    """
    Analyze and compare multiple stocks with enhanced features
    
    Args:
        tickers: List of stock ticker symbols
        num_articles: Number of news articles per stock
        chart_period: Time period for charts
    """
    results = {}
    
    for ticker in tickers:
        print(f"\n{'='*80}\n")
        result = analyze_stock_enhanced(ticker, num_articles, chart_period)
        results[ticker] = result
        print(f"\n{'='*80}\n")
    
    return results

---

## 💡 Usage Examples

Run the cells below to analyze stocks!

### Example 1: Analyze TSLA

In [11]:
# Analyze Tesla with enhanced features
tsla_analysis = analyze_stock_enhanced('TSLA', num_articles=5, chart_period='1mo')

🔍 Analyzing TSLA...

📰 Fetching latest news for TSLA...
✓ Found 5 articles

📊 Fetching stock price data...
✓ Retrieved 1mo price data

🤖 Generating AI summary with references...
✓ Summary generated




📈 Stock Price Chart:


### Example 2: Analyze TSM

In [12]:
# Analyze Taiwan Semiconductor with enhanced features
tsm_analysis = analyze_stock_enhanced('TSM', num_articles=5, chart_period='1mo')

🔍 Analyzing TSM...

📰 Fetching latest news for TSM...
✓ Found 5 articles

📊 Fetching stock price data...
✓ Retrieved 1mo price data

🤖 Generating AI summary with references...
✓ Summary generated




📈 Stock Price Chart:


### Example 3: Compare Multiple Stocks

In [13]:
# Compare TSLA and TSM side by side
results = compare_multiple_stocks_enhanced(['TSLA', 'TSM'], num_articles=5, chart_period='1mo')



🔍 Analyzing TSLA...

📰 Fetching latest news for TSLA...
✓ Found 5 articles

📊 Fetching stock price data...
✓ Retrieved 1mo price data

🤖 Generating AI summary with references...
✓ Summary generated




📈 Stock Price Chart:






🔍 Analyzing TSM...

📰 Fetching latest news for TSM...
✓ Found 5 articles

📊 Fetching stock price data...
✓ Retrieved 1mo price data

🤖 Generating AI summary with references...
✓ Summary generated




📈 Stock Price Chart:


### Example 4: Custom Analysis

In [14]:
# Analyze any stock you want with custom parameters
# Change the ticker, number of articles, and chart period

# analyze_stock_enhanced('AAPL', num_articles=10, chart_period='3mo')
# analyze_stock_enhanced('NVDA', num_articles=7, chart_period='6mo')
analyze_stock_enhanced('GOOGL', num_articles=5, chart_period='1y')

🔍 Analyzing GOOGL...

📰 Fetching latest news for GOOGL...
✓ Found 5 articles

📊 Fetching stock price data...
✓ Retrieved 1y price data

🤖 Generating AI summary with references...
✓ Summary generated




📈 Stock Price Chart:


{'ticker': 'GOOGL',
 'articles': [{'ticker': 'GOOGL',
   'title': 'Investor George Soros Increases Stake in Alphabet Stock (GOOGL) by 2,300% - TipRanks',
   'link': 'https://news.google.com/rss/articles/CBMioAFBVV95cUxQVmxmX1p2ZGFWalpuNW91YWZIT1FKdE5qLUJBaE5GeUMzTUhra0o0bHk0ejMxb1NPcHlhZURCVFlVNXVDUHZaRmsySm80QXNqWWZlT19HWGlCWXZYREpZQUxtSmRKLVVlSzNSNHJ0a2ctVk52NzltNUtZLWNQZFpuLTBPX3QzcEFlSGZldVBjN1ctZ0ZfbllNWEswVkxWVEI0?oc=5',
   'published': 'Thu, 20 Nov 2025 21:22:16 GMT',
   'summary': '<a href="https://news.google.com/rss/articles/CBMioAFBVV95cUxQVmxmX1p2ZGFWalpuNW91YWZIT1FKdE5qLUJBaE5GeUMzTUhra0o0bHk0ejMxb1NPcHlhZURCVFlVNXVDUHZaRmsySm80QXNqWWZlT19HWGlCWXZYREpZQUxtSmRKLVVlSzNSNHJ0a2ctVk52NzltNUtZLWNQZFpuLTBPX3QzcEFlSGZldVBjN1ctZ0ZfbllNWEswVkxWVEI0?oc=5" target="_blank">Investor George Soros Increases Stake in Alphabet Stock (GOOGL) by 2,300%</a>&nbsp;&nbsp;<font color="#6f6f6f">TipRanks</font>'},
  {'ticker': 'GOOGL',
   'title': "Amazon And Google's 'OpenAI Dilemma' Sparks Debate 